In [119]:
import pandas as pd 

In [120]:
df = pd.read_csv('../dataSet/datasetTitle/titleLabel1000.csv')
# df['title'].iloc[1]

In [121]:
df.dropna(inplace=True)

In [122]:
df['label'] = df['label'].astype("int64")


# Case Folding

In [123]:
import re
import string
def casefolding(kalimat):
    # kalimat = kalimat.translate(str.maketrans(' ',' ',string.punctuation))
    kalimat = kalimat.strip()
    kalimat = kalimat.lower()
    kalimat = re.sub(r'[|?|$|.|!_:")(-+,)]','', kalimat)
    # kalimat = re.sub('<.*>',' ',kalimat)
    # kalimat = re.sub('[^a-zA-Z]',' ', kalimat)
    # kalimat = re.sub(r"\b[a-aZ-Z]\b"," ", kalimat)
    # kalimat = re.sub(r'\b\w\b', '', kalimat)
    # kalimat = re.sub("\n"," ", kalimat)
    return kalimat

In [124]:

df['title'] = df['title'].apply(casefolding)
df['title'].iloc[1]

'manusia diterkam beruang'

# Tokenizing

In [125]:
from nltk.tokenize import RegexpTokenizer
regexp = RegexpTokenizer(r'\w+|$[0-9]+|\S')

In [126]:
token = df["title"].apply(regexp.tokenize)
df.insert(df.columns.get_loc("label"),"Token", token)

In [127]:
df['Token'].iloc[1]

['manusia', 'diterkam', 'beruang']

In [128]:
from nltk.tag import CRFTagger

In [129]:
crft = CRFTagger()
crft.set_model_file('../resource/all_indo_man_tag_corpus_model.crf.tagger')

In [130]:
tag = crft.tag_sents(df.Token)

In [131]:
df.insert(df.columns.get_loc("label"),"tag", tag)


In [132]:
df['tag'].iloc[24]

[('museum', 'NN'),
 ('subang', 'NN'),
 ('masih', 'MD'),
 ('kurang', 'RB'),
 ('pengunjung', 'VB')]

# StopWord

In [133]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import CRFTagger
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("CRF++-0.58-cp37-cp37m-win_amd64.whl")
nltk.download("CRF++-0.58-cp38-cp38-win_amd64.whl")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AdmiN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AdmiN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Error loading CRF++-0.58-cp37-cp37m-win_amd64.whl: Package
[nltk_data]     'CRF++-0.58-cp37-cp37m-win_amd64.whl' not found in
[nltk_data]     index
[nltk_data] Error loading CRF++-0.58-cp38-cp38-win_amd64.whl: Package
[nltk_data]     'CRF++-0.58-cp38-cp38-win_amd64.whl' not found in
[nltk_data]     index


False

In [134]:
def remove_stop_word_by_tag(text):
    # Tokenisasi kalimat
    # sentences = sent_tokenize(text, language='indonesian')
    sentence = text
    result = []
    
    # Membuat CRF Tagger
    tagger = CRFTagger()
    tagger.set_model_file('../resource/all_indo_man_tag_corpus_model.crf.tagger')  # Gantilah path dengan lokasi model POS tagger untuk bahasa Indonesia
    
    for sentence in sentence:
        # Tokenisasi kata-kata dalam kalimat
        words = word_tokenize(sentence)
        
        stopword = stopwords.words('indonesian')
        txt_stopword = pd.read_csv('../resource/stopword.txt', names=['stopword'], header=None)
        stopword.extend(['wkwk','hahahaha','haha','yang','yoi','yoyoy', 'mm', 'mk', 'bandung', 'subang','wartakinico','galamedianews','rp','$'])
        stopword.extend(txt_stopword["stopword"][0].split('\n'))
        stopword = set(stopword)
        # Menghapus stop words berdasarkan tag
        tagged_words = tagger.tag(words)
        filtered_words = [word for word, tag in tagged_words if word.lower() not in stopword or tag.startswith(('NN', 'RB'))]
        
        # Menggabungkan kata-kata yang tersaring untuk membentuk kalimat baru
        filtered_sentence = ' '.join(filtered_words)
        
        # Menambahkan kalimat yang telah dihapus stop words ke hasil akhir
        result.append(filtered_sentence)
    
    # Menggabungkan kalimat menjadi teks lengkap
    final_text = ' '.join(result)
    
    return final_text

In [135]:
stopwordByTag = df['Token'].apply(remove_stop_word_by_tag)
df.insert(df.columns.get_loc("label"),"stopwordByTag", stopwordByTag)

In [136]:
print(df['title'].iloc[982])

ditanya soal pengembalian kerugian negara dinas pupr bangkalan sebut bpk kurang update


In [137]:
print(df['stopwordByTag'].iloc[982])


ditanya soal pengembalian kerugian negara dinas pupr bangkalan  bpk kurang update


In [138]:
print(df['tag'].iloc[982])


[('ditanya', 'VB'), ('soal', 'NN'), ('pengembalian', 'NN'), ('kerugian', 'NN'), ('negara', 'NN'), ('dinas', 'NN'), ('pupr', 'NN'), ('bangkalan', 'NN'), ('sebut', 'VB'), ('bpk', 'NN'), ('kurang', 'RB'), ('update', 'FW')]


In [139]:
import nltk as nk 
nk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AdmiN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [140]:
# from nltk.corpus import stopwords
# stopword = stopwords.words('indonesian')
# txt_stopword = pd.read_csv('../resource/stopword.txt', names=['stopword'], header=None)
# stopword.extend(['wkwk','hahahaha','haha','yang','yoi','yoyoy', 'mm', 'mk', 'bandung', 'subang','wartakinico','galamedianews','rp','$'])
# stopword.extend(txt_stopword["stopword"][0].split('\n'))
# stopword = set(stopword)

# def stopwords(text):
#     text = [word for word in text if word not in stopword]
#     return text

In [141]:
# stopword = df['Token'].apply(stopwords)
# df.insert(df.columns.get_loc("label"),"stopword", stopword)

In [142]:
# df[['title', 'stopword']].iloc[24]

# Stemming

### Pilih Salah satu dari metode stemming di bawah, Menggunakan join atau tanpa menggunakan join

In [143]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [144]:
# Opsi Pertama: Melakukan stemming dan meReturn hasilnya dengan join 
def stemmingWithJoin(konten):
    words = word_tokenize(konten)
    do = []
    for w in words:
        dt = stemmer.stem(w)
        do.append(dt)
    
    d_clean = []
    d_clean = " ".join(do)
    return d_clean

In [145]:
# Opsi Kedua : Melakukan stemming dengan tidak mereturnnya dengan join

# def stemmed_wrapper(term):
#     return stemmer.stem(term)

# term_dict = {}

# kata = df['stopword']
# for doc in kata:
#     for term in doc:
#         if term not in term_dict:
#             term_dict[term] = ''
    
    
# for term in term_dict:
#     term_dict[term] = stemmed_wrapper(term)


# def get_stemmed_term(doc):
#     return [term_dict[term] for term in doc]

In [146]:
# Menggunakan opsi pertama
stemmed = df['stopwordByTag'].apply(stemmingWithJoin)
df.insert(df.columns.get_loc("label"),"stemmed", stemmed)


In [147]:
#Menggunakan Opsi ke dua

# stemmed = df['stopword'].apply(get_stemmed_term)
# df.insert(df.columns.get_loc("label"),"stemmed", stemmed)



In [148]:
df.head()

,title,Token,tag,stopwordByTag,stemmed,label
0,ngatiyana terjun langsung menemui warga menamp...,"[ngatiyana, terjun, langsung, menemui, warga, ...","[(ngatiyana, SC), (terjun, VB), (langsung, RB)...",ngatiyana terjun langsung menemui warga menamp...,ngatiyana terjun langsung temu warga tampung a...,1
1,manusia diterkam beruang,"[manusia, diterkam, beruang]","[(manusia, NN), (diterkam, NN), (beruang, NN)]",manusia diterkam beruang,manusia terkam beruang,0
2,puluhan rumah di perum griya zavira cilawu ter...,"[puluhan, rumah, di, perum, griya, zavira, cil...","[(puluhan, CD), (rumah, NN), (di, IN), (perum,...",puluhan rumah perum griya zavira cilawu teran...,puluh rumah perum griya zavira cilawu ancam ba...,2
3,prakiraan cuaca kota-kota besar di indonesia u...,"[prakiraan, cuaca, kota, -, kota, besar, di, i...","[(prakiraan, NN), (cuaca, NN), (kota, NN), (-,...",prakiraan cuaca kota - kota indonesia senin...,prakira cuaca kota - kota indonesia senin 25 j...,0
4,kurang dari 24 jam pelaku pembunuh remaja dici...,"[kurang, dari, 24, jam, pelaku, pembunuh, rema...","[(kurang, RB), (dari, IN), (24, CD), (jam, NN)...",kurang 24 jam pelaku pembunuh remaja diciduk ...,kurang 24 jam laku bunuh remaja ciduk saat cob...,2


In [149]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


In [150]:
def lemmatize_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(sentence)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    lemmatized_sentence = ' '.join(lemmatized_words)
    return lemmatized_sentence



In [151]:
llema = df['stopwordByTag'].apply(lemmatize_sentence)
df.insert(df.columns.get_loc("label"),"lemma", llema)


# Export File hasil Preprocessing

In [152]:
df.to_csv('../dataSet/datasetTitle/titleClean1000_3.csv', header=True , index=False)

In [153]:
url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format("root", "", "127.0.0.1", 3306, 'db_sma2')


In [154]:
df['ceked'] = df['title'] == df['stopwordByTag']

In [155]:
df.head()

,title,Token,tag,stopwordByTag,stemmed,lemma,label,ceked
0,ngatiyana terjun langsung menemui warga menamp...,"[ngatiyana, terjun, langsung, menemui, warga, ...","[(ngatiyana, SC), (terjun, VB), (langsung, RB)...",ngatiyana terjun langsung menemui warga menamp...,ngatiyana terjun langsung temu warga tampung a...,ngatiyana terjun langsung menemui warga menamp...,1,True
1,manusia diterkam beruang,"[manusia, diterkam, beruang]","[(manusia, NN), (diterkam, NN), (beruang, NN)]",manusia diterkam beruang,manusia terkam beruang,manusia diterkam beruang,0,True
2,puluhan rumah di perum griya zavira cilawu ter...,"[puluhan, rumah, di, perum, griya, zavira, cil...","[(puluhan, CD), (rumah, NN), (di, IN), (perum,...",puluhan rumah perum griya zavira cilawu teran...,puluh rumah perum griya zavira cilawu ancam ba...,puluhan rumah perum griya zavira cilawu teranc...,2,False
3,prakiraan cuaca kota-kota besar di indonesia u...,"[prakiraan, cuaca, kota, -, kota, besar, di, i...","[(prakiraan, NN), (cuaca, NN), (kota, NN), (-,...",prakiraan cuaca kota - kota indonesia senin...,prakira cuaca kota - kota indonesia senin 25 j...,prakiraan cuaca kota - kota indonesia senin 25...,0,False
4,kurang dari 24 jam pelaku pembunuh remaja dici...,"[kurang, dari, 24, jam, pelaku, pembunuh, rema...","[(kurang, RB), (dari, IN), (24, CD), (jam, NN)...",kurang 24 jam pelaku pembunuh remaja diciduk ...,kurang 24 jam laku bunuh remaja ciduk saat cob...,kurang 24 jam pelaku pembunuh remaja diciduk s...,2,False


In [156]:
df[['title','stopwordByTag','ceked','lemma','stemmed',]].to_sql('testing_0.1.1', con=url, index=False, if_exists = 'replace')


1000